[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/seagarwu/mynotebooks/blob/main/shioaji/shioaji_onlinedoc/shioaji-tutorial_user_guide.ipynb)

# Tutorial - User Guide

## Login

### Login Shioaji

In [ ]:
import shioaji as sj
import getpass as gp
user_id, user_pass = gp.getpass("USER ID: "), gp.getpass("USER PASS: ")
api = sj.Shioaji()
api.login(
    person_id=user_id,
    passwd=user_pass,
    contracts_cb=lambda security_type: print(f"{repr(security_type)} fetch done.")
)

Response Code: 0 | Event Code: 0 | Info: host '203.66.91.161:80', hostname '203.66.91.161:80' IP 203.66.91.161:80 (host 1 of 1) (host connection attempt 1 of 1) (total connection attempt 1 of 1) | Event: Session up


[FutureAccount(person_id='R120732106', broker_id='F002002', account_id='8070913', signed=True, username='吳鼎榮'),
 FutureAccount(person_id='R120732106', broker_id='F002002', account_id='8071080', signed=True, username='吳鼎榮'),
 FutureAccount(person_id='R120732106', broker_id='F002002', account_id='8071378', signed=True, username='吳鼎榮')]

In [ ]:
import tkinter as tk
from tkinter import filedialog

root = tk.Tk()
root.withdraw()
file_path = filedialog.askopenfilename()

api.activate_ca(
    ca_path = file_path,
    ca_passwd = user_pass,
    person_id = user_id,
)

True

####  Default Account

##### List Accounts
There are 2 ways you can get your accounts:
1. api.login
2. api.list_accounts

In [ ]:
# api.login
accounts = api.login(
    person_id="YOUR_ID", 
    passwd="YOUR_PASSWORD", 
)

In [ ]:
# api.list_accounts
accounts = api.list_accounts()
accounts

[FutureAccount(person_id='R120732106', broker_id='F002002', account_id='8070913', signed=True, username='吳鼎榮'),
 FutureAccount(person_id='R120732106', broker_id='F002002', account_id='8071080', signed=True, username='吳鼎榮'),
 FutureAccount(person_id='R120732106', broker_id='F002002', account_id='8071378', signed=True, username='吳鼎榮')]

- Note that if your account hasn't been signed yet, ex: ACCOUNT_ID_2, ACCOUNT_ID_3, please sign the document first.

##### Default Account

In [ ]:
# Stock default account
print(api.stock_account)

# Futures default account
print(api.futopt_account)

None
person_id='R120732106' broker_id='F002002' account_id='8070913' signed=True username='吳鼎榮'


##### Default Account in Order
In `Order` object, you need to specify which account you want to place order. For more information about `Order`, please refer to [Stock Order](#stocks) and [Futures Order](https://sinotrade.github.io/tutor/order/FutureOption/).

In [ ]:
# stock order
order = api.Order(
    price=12, 
    quantity=1, 
    action=sj.constant.Action.Buy, 
    price_type=sj.constant.StockPriceType.LMT, 
    order_type=sj.constant.TFTOrderType.ROD, 
    order_lot=sj.constant.TFTStockOrderLot.Common, 
    account=api.stock_account
)

# futures order
order = api.Order(
    action=sj.constant.Action.Buy,
    price=10200,
    quantity=1,
    price_type=sj.constant.StockPriceType.LMT,
    order_type=sj.constant.FuturesOrderType.ROD, 
    octype=sj.constant.FuturesOCType.Auto,
    account=api.futopt_account
)

##### Set Default Account

In [ ]:
# stock default account
api.set_default_account(accounts[-1])
print(api.stock_account)

# futures default account
api.set_default_account(accounts[0])
print(api.futopt_account)

None
person_id='R120732106' broker_id='F002002' account_id='8070913' signed=True username='吳鼎榮'


#### set subscribe trade callback
There are 2 ways you can set subscribe trade callback:
1. api.login
2. api.subscribe_trade and api.unsubscribe_trade

In [ ]:
# api.login
api.login(
    person_id="YOUR_ID", 
    passwd="YOUR_PASSWORD", 
    subscribe_trade=False,
    )

In [ ]:
# api.subscribe_trade
account = api.futopt_account
print(account)
api.subscribe_trade(account)

person_id='R120732106' broker_id='F002002' account_id='8070913' signed=True username='吳鼎榮'


True

In [ ]:
# api.unsubscribe_trade
api.unsubscribe_trade(account)

**Note that**: * Default subscribe trade callback is True.

### Logout Shioaji
Logout funciton will close the connection between the client and the server.<br>
In order to provide high quality services, starting from 2021/08/06, we've limit the number of connections used. It's a good practice to logout or to terminate the program when it is not in use.

In [ ]:
api.logout()

True

## Contract

### Login Fetch Contracts

#### [First, you need login.](#login)
After login success we will start to fetch all kind of contract but fetching will not block other action. So how to know the fetch action is done ? We have status of contracts download that you can use `Contracts.status`.

**contracts_timeout**<br>
If you set contracts_timeout inside `login` from 0 to 10000, it will block the fetch and wait 10 second until 
the contract is back.<br>
If you set contract_cb (contract callback) as print, it will print "SecurityType.Index, SecurityType.Stock, 
SecurityType.Future, SecurityType.Option " until the contract is back.


In [ ]:
api.login(
    person_id='YOUR_ID', 
    passwd='YOUR_PASSWORD', 
    contracts_timeout=10000,
    contracts_cb=print,
) 

1. [Contracts的資料目前分為4大類](https://ithelp.ithome.com.tw/articles/10279819)：指數Index、股票Stock、期貨Future跟Option選擇權，在執行login()時，在登入成功後，shioaji會開始下載及初始化Contract資料。而contracts_cb這個參數，就是當上面4大類的Contract資料下載及初始完成後，就會執行contracts_cb中所傳入的function。
2. 當api.login()時會以非同步方式抓取contracts並放到api.Contracts，但這需要時間，如果如同網頁所測試，在還沒抓完就直接讀取api.Contracts內容的話，會造成內容不完整的問題。我想這是contracts_timeout存在的原因，或者可以用網頁介紹的方法: 使用event.wait()等待抓取完成。

In [ ]:
# 網頁介紹: 遇到api.Contracts抓取不完整的程式碼
# Contracts.Stocks資料下載完成後所輸出的「SecurityType.Stock」訊息，卻是在for loop迴圈完成之後，
# 這表示我們抓的Contracts.Stocks其實是不完整的
# 實測時很多時候不會跑contracts_cb，不知道為什麼?!
import shioaji as sj
import pandas as pd
import getpass as gp

api = sj.Shioaji()
user_id, user_pass = gp.getpass("USER ID: "), gp.getpass("USER PASS: ")

api.login(
    person_id=user_id, 
    passwd=user_pass,
    contracts_cb=lambda security_type: print(f"{repr(security_type)} fetch done.") #設定callback為print，即完成初始化時輸出至console
)
print('api.login is done...') #輸出目前執行的步驟至console
stock_list = []

for exchange in api.Contracts.Stocks:
    for stock in exchange:
        stock_list.append({**stock})

print('for loop is done...') #輸出目前執行的步驟至console
print(f'len(stock_list) is :{len(stock_list)}')

api.logout()

Response Code: 0 | Event Code: 0 | Info: host '203.66.91.161:80', hostname '203.66.91.161:80' IP 203.66.91.161:80 (host 1 of 1) (host connection attempt 1 of 1) (total connection attempt 1 of 1) | Event: Session up
api.login is done...
for loop is done...
len(stock_list) is :41214


True

In [ ]:
# 網頁介紹: 遇到api.Contracts抓取不完整的程式碼 (解決方法。但實測時很多時候不會跑contracts_cb，就卡在event.wait()，不知道為什麼?!)
import os
import shioaji as sj
from shioaji.constant import SecurityType #匯入SecurityType常數
import pandas as pd
import threading #匯入threading模組
import getpass as gp

event = threading.Event() #宣告event
api = sj.Shioaji()
user_id, user_pass = gp.getpass("USER ID: "), gp.getpass("USER PASS: ")

def my_cb(security_type):
    print(f"{repr(security_type)} fetch done.")
    #當Contracts.Stocks下載完成時，輸出訊息並執行event.set()
    if security_type == SecurityType.Option:  # Option看起來才是最後一個
        print('final fetch done.')
        event.set() #讓原本wait的程式繼續執行

api.login(
    person_id=user_id, 
    passwd=user_pass,
    contracts_cb=my_cb #指定callback為my_cb
)
print('api.login is done...')
event.wait() #api.login執行完後，讓程式先進入等待
stock_list = []
print('start for loop...')
for exchange in api.Contracts.Stocks:
    for stock in exchange:
        stock_list.append({**stock})

print('for loop is done...')
print(f'len(stock_list) is :{len(stock_list)}')
df = pd.DataFrame(stock_list)
print(len(df))
print(len(stock_list))
df.to_csv('stock_list.csv', index=False, encoding="utf_8_sig")
print('df.to_csv is done...')

api.logout()

Response Code: 0 | Event Code: 0 | Info: host '203.66.91.161:80', hostname '203.66.91.161:80' IP 203.66.91.161:80 (host 1 of 1) (host connection attempt 1 of 1) (total connection attempt 1 of 1) | Event: Session up
<SecurityType.Index: 'IND'> fetch done.
<SecurityType.Future: 'FUT'> fetch done.
api.login is done...
<SecurityType.Stock: 'STK'> fetch done.
<SecurityType.Option: 'OPT'> fetch done.
final fetch done.
start for loop...
for loop is done...
len(stock_list) is :41214
41214
41214
df.to_csv is done...


True

**fetch_contract**<br>
If fetch_contract inside `login` is set to TRUE, it will download contract again.

In [ ]:
api.login(
    person_id='YOUR_ID', 
    passwd='YOUR_PASSWORD', 
    fetch_contract=True,
) 

### Contracts

In [ ]:
api.Contracts

Contracts(Indexs=(OTC, TSE), Stocks=(OES, OTC, TSE), Futures=(BRF, BTF, CAF, CBF, CCF, CDF, CEF, CFF, CGF, CHF, CJF, CKF, CLF, CMF, CNF, CQF, CRF, CSF, CUF, CWF, CYF, CZF, DCF, DD1, DDF, DEF, DFF, DGF, DHF, DIF, DJF, DKF, DLF, DNF, DOF, DPF, DQF, DSF, DVF, DWF, DXF, DYF, DZF, E4F, EEF, EG1, EGF, EHF, EKF, EMF, EPF, ERF, EXF, EYF, EZF, F1F, FB1, FBF, FCF, FEF, FFF, FGF, FKF, FNF, FQF, FRF, FTF, FVF, FW1, FWF, FXF, FYF, FZF, G2F, GAF, GCF, GDF, GHF, GIF, GJF, GLF, GMF, GNF, GOF, GRF, GTF, GUF, GWF, GXF, GZF, HAF, HBF, HCF, HHF, HIF, HLF, HOF, HSF, IAF, IHF, IIF, IJF, IMF, IOF, IPF, IQF, IRF, ITF, IXF, IYF, IZF, JBF, JFF, JMF, JNF, JPF, JSF, JWF, JZF, KAF, KBF, KCF, KDF, KEF, KFF, KGF, KIF, KKF, KLF, KOF, KPF, KSF, KUF, KWF, LBF, LCF, LEF, LIF, LMF, LOF, LQF, LRF, LTF, LUF, LVF, LWF, LXF, LYF, MAF, MBF, MJF, MKF, MPF, MQF, MVF, MX2, MXF, MYF, NAF, NBF, NCF, NDF, NEF, NGF, NIF, NJF, NLF, NMF, NOF, NQF, NSF, NUF, NVF, NWF, NXF, NYF, NZF, OAF, OBF, OCF, ODF, OEF, OHF, OJF, OKF, OLF, OMF, OOF

If your contracts have been downloaded, will see the output like above. If part of that kind contract download not yet, It will still be `None`.

#### Stock Contract

In [ ]:
contract_2890 = api.Contracts.Stocks["2890"]
contract_2890

Stock(exchange=<Exchange.TSE: 'TSE'>, code='2890', symbol='TSE2890', name='永豐金', category='17', unit=1000, limit_up=20.9, limit_down=17.1, reference=19.0, update_date='2022/04/12', day_trade=<DayTrade.Yes: 'Yes'>)

> <details><summary>Attributes:</summary>
>
> exchange (Exchange): Attributes of industry.<br>
>     {OES, OTC, TSE ...etc}<br>
> code (str): Id.<br>
> symbol (str): Symbol.<br>
> name (str): Name.<br>
> category (str): Category.<br>
> limit_up (float): Limit up.<br>
> limit_down (float): Limit down.<br>
> reference (float): Reference price.<br>
> update_date (str): Update date.<br>
> margin_trading_balance (int): Margin trading balance.<br>
> short_selling_balance (int): Short selling balance.<br>
> day_trade (DayTrade): Day trade.<br>
>     {Yes, No, OnlyBuy}<br>
</details>

#### Futures Contract

In [ ]:
contract_txf = api.Contracts.Futures.MXF.MXF202204  # 等同 api.Contracts.Futures['MXFD2']
contract_txf

Future(code='MXFD2', symbol='MXF202204', name='小型臺指04', category='MXF', delivery_month='202204', delivery_date='2022/04/20', underlying_kind='I', unit=1, limit_up=18700.0, limit_down=15300.0, reference=17000.0, update_date='2022/04/13')

> <details><summary>Attributes:</summary>
>
> code (str): Id.<br>
> symbol (str): Symbol.<br>
> name (str): Name.<br>
> category (str): Category.<br>
> limit_up (float): Limit up.<br>
> limit_down (float): Limit down.<br>
> reference (float): Reference price.<br>
> update_date (str): Update date.<br>
> delivery_month (str): Delivery Month.<br>
> underlying_kind (str): Underlying Kind.<br>
</details>

#### Options Contract

##### Options object show all avaiable options product.

In [ ]:
# 辨識號碼一覽表: https://isin.twse.com.tw/isin/C_public.jsp?strMode=6
# 商品簡稱及英文代碼: https://www.taifex.com.tw/cht/4/contractName1
# Get options by code.
api.Contracts.Options

(CAO, CBO, CCO, CDO, CEO, CFO, CGO, CHO, CJO, CKO, CLO, CMO, CNO, CQO, CRO, CSO, CZO, DCO, DEO, DFO, DGO, DHO, DJO, DKO, DLO, DNO, DOO, DPO, DQO, DSO, DVO, DWO, DXO, GIO, GXO, HCO, IJO, LOO, NYO, NZO, OAO, OBO, OCO, OJO, OKO, OOO, OZO, QBO, RHO, RTO, TEO, TFO, TGO, TX1, TXO)

In [ ]:
api.Contracts.Options.TX1

TX1(TX1202205016000P, TX1202205016200P, TX1202205018100P, TX1202205016250P, TX1202205015850C, TX1202205017400C, TX1202205018200C, TX1202205018000C, TX1202205015600C, TX1202205016650P, TX1202205016550P, TX1202205015500P, TX1202205014900P, TX1202205015700C, TX1202205017050C, TX1202205016400C, TX1202205016900P, TX1202205016650C, TX1202205015600P, TX1202205017100P, TX1202205016400P, TX1202205017500P, TX1202205018000P, TX1202205016550C, TX1202205014800C, TX1202205014600C, TX1202205017000C, TX1202205018100C, TX1202205017500C, TX1202205015000P, TX1202205018400P, TX1202205016850C, TX1202205014800P, TX1202205018200P, TX1202205017900C, TX1202205018300P, TX1202205016250C, TX1202205017200P, TX1202205016150C, TX1202205016100P, TX1202205016300C, TX1202205016600C, TX1202205014700C, TX1202205014600P, TX1202205016800C, TX1202205016500C, TX1202205015300P, TX1202205016750C, TX1202205016050P, TX1202205016350P, TX1202205017400P, TX1202205016000C, TX1202205014700P, TX1202205015900P, TX1202205018400C, TX1202

##### Get options by symbol.

In [ ]:
api.Contracts.Options.TX1.TX1202205014700C

Option(code='TX114700E2', symbol='TX1202205014700C', name='臺指選擇權05W1月 14700C', category='TX1', delivery_month='202205', delivery_date='2022/05/04', strike_price=14700.0, option_right=<OptionRight.Call: 'C'>, underlying_kind='I', unit=1, limit_up=3350.0, limit_down=69.0, reference=1710.0, update_date='2022/04/29')

##### Get options by code.

In [ ]:
api.Contracts.Options["TX114700E2"]

Option(code='TX114700E2', symbol='TX1202205014700C', name='臺指選擇權05W1月 14700C', category='TX1', delivery_month='202205', delivery_date='2022/05/04', strike_price=14700.0, option_right=<OptionRight.Call: 'C'>, underlying_kind='I', unit=1, limit_up=3350.0, limit_down=69.0, reference=1710.0, update_date='2022/04/29')

> Get put options with readable way.<br>
> 
> <details><summary>Attributes:</summary>
> code (str): Id.<br>
> symbol (str): Symbol.<br>
> name (str): Name.<br>
> category (str): Category.<br>
> limit_up (float): Limit up.<br>
> limit_down (float): Limit down.<br>
> reference (float): Reference price.<br>
> update_date (str): Update date.<br>
> delivery_month (str): Delivery Month.<br>
> strike_price (int or float): Strike Price.<br>
> option_right (OptionRight): Option Right.<br>
> underlying_kind (str): Underlying Kind.<br>
</details>

#### Index Contract

In [ ]:
api.Contracts.Indexs.TSE

TSE(TSE001, TSE002, TSE003, TSE004, TSE005, TSE006, TSE008, TSE010, TSE015, TSE016, TSE017, TSE018, TSE019, TSE020, TSE021, TSE022, TSE023, TSE024, TSE025, TSE026, TSE027, TSE028, TSE029, TSE030, TSE031, TSE032, TSE033, TSE034, TSE035, TSE036, TSE037, TSE038, TSE039, TSE040, TSE041, TSE042, TSE043, TSE053, TSE054, TSE055, TSE056, TSE057, TSE058, TSE059, TSE060, TSE061, TSE062, TSE064, TSE065, TSE066, TSE067, TSE068, TSE069, TSE070, TSE071, TSE072, TSE073, TSE074, TSE075, TSE076, TSE077, TSE078, TSE079, TSE081, TSE082, TSE083, TSE084, TSE085, TSE086, TSE087, TSE088, TSE089, TSE090, TSE091, TSE092, TSE093, TSE094, TSE095, TSE096, TSE097, TSE098, TSE099)

In [ ]:
api.Contracts.Indexs.TSE.TSE001

Index(exchange=<Exchange.TSE: 'TSE'>, code='001', symbol='TSE001', name='加權指數')

> Attributes:<br>
>
> exchange (Exchange): Attributes of industry.<br>
>     {OES, OTC, TSE ...etc}<br>
> code (str): Id.<br>
> symbol (str): Symbol.<br>
> name (str): Name.<br>
> 
> This contract is not avaiable to place order, but allow to subscribe the quote. It will be disuccess at next topic.

### Conclusion
Contract object will be used by a lot of place like place order and subscribe quote etc... So Keep in mind how to get the contract you want to use.

---
### Prev: [Quick Start](shioaji-quick_start.ipynb) Next: [Market Data / Streaming Market Data](shioaji-market_data-streaming_market_data.ipynb)